---
## Setup and Imports

In [1]:
import os
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Core imports
from src.core.models import LoopConfig, FinalResult
from src.core.rule_memory import RuleMemory, FluentEntry
from src.loop.orchestrator import LoopOrchestrator
from src.prompts.builder import MSAPromptBuilder, HARPromptBuilder
from src.llm.factory import ProviderFactory
from src.simlp.client import SimLPClient

# Utilities
import pandas as pd
from datetime import datetime

print("✓ All imports successful!")
print(f"Working directory: {Path.cwd()}")
print(f"Project root: {project_root}")

✓ All imports successful!
Working directory: /Users/gphome/Desktop/projects/thesis-ds/feedback-loop/notebooks
Project root: /Users/gphome/Desktop/projects/thesis-ds/feedback-loop


---
## 1. Understanding RuleMemory

The **RuleMemory** class is a key-value store for RTEC fluent rules:
- **Key**: Fluent name (e.g., `"gap"`, `"trawlSpeed"`)
- **Value**: `FluentEntry` containing description, rules, score, and metadata

This allows the orchestrator to:
- Store best rules after each run
- Retrieve prerequisite rules for dependent activities
- Build complex activity hierarchies

In [2]:
# Create an empty memory instance
memory = RuleMemory()

print(f"Memory initialized: {len(memory)} entries")
print(f"Is empty? {memory.is_empty()}")

Memory initialized: 0 entries
Is empty? True


---
## 2. Manual Memory Operations

Before using with the orchestrator, let's explore basic memory operations:

In [3]:
# Add a fluent entry manually
memory.add_entry(
    name="gap",
    description="Vessel communication gap - when AIS signal is lost",
    rules="""
initiatedAt(gap(Vessel)=nearPort, T) :-
    happensAt(gap_start(Vessel), T),
    holdsAt(withinArea(Vessel, nearPort)=true, T).

terminatedAt(gap(Vessel)=nearPort, T) :-
    happensAt(gap_end(Vessel), T).
""".strip(),
    score=0.95,
    metadata={
        "domain": "MSA",
        "prerequisites": [],
        "fluent_type": "simple"
    }
)

print(f"✓ Added 'gap' entry")
print(f"Memory now has {len(memory)} entries")

✓ Added 'gap' entry
Memory now has 1 entries


In [4]:
# Retrieve the entry
entry = memory.get("gap")

print(f"Name: {entry.name}")
print(f"Description: {entry.description}")
print(f"Score: {entry.score}")
print(f"Domain: {entry.metadata.get('domain')}")
print(f"\nRules:\n{entry.rules}")

Name: gap
Description: Vessel communication gap - when AIS signal is lost
Score: 0.95
Domain: MSA

Rules:
initiatedAt(gap(Vessel)=nearPort, T) :-
    happensAt(gap_start(Vessel), T),
    holdsAt(withinArea(Vessel, nearPort)=true, T).

terminatedAt(gap(Vessel)=nearPort, T) :-
    happensAt(gap_end(Vessel), T).


In [6]:
# Check if entries exist
print(f"Contains 'gap': {memory.contains('gap')}")
print(f"Contains 'nonexistent': {memory.contains('nonexistent')}")


Contains 'gap': True
Contains 'nonexistent': False


---
## 3. Formatted Rule Retrieval

The memory provides formatted output suitable for prompt injection:

In [7]:
# Get formatted rules for multiple fluents
formatted = memory.get_formatted_rules(
    names=["gap"],
    include_description=True
)

print("Formatted output for prompts:")
print(formatted)

Formatted output for prompts:
% === gap ===
% Description: Vessel communication gap - when AIS signal is lost
initiatedAt(gap(Vessel)=nearPort, T) :-
    happensAt(gap_start(Vessel), T),
    holdsAt(withinArea(Vessel, nearPort)=true, T).

terminatedAt(gap(Vessel)=nearPort, T) :-
    happensAt(gap_end(Vessel), T).


---
## 4. LoopOrchestrator with Memory

Now let's integrate memory with the orchestrator. The orchestrator will:
1. Check memory for prerequisite rules
2. Inject them into prompts
3. Store the best rules after completion

### 4.1 Configure Components

In [ ]:
# Configuration
config = LoopConfig(
    provider="openai",
    objective="Generate accurate RTEC rules for maritime activities",
    max_iterations=3,
    convergence_threshold=0.90,
    batch_size=1,
    retry_limit=3
)

# Prompt builder
prompt_builder = MSAPromptBuilder()

# LLM provider (requires OPENAI_API_KEY in environment)
llm_provider = ProviderFactory.create("openai", api_key=os.getenv("OPENAI_API_KEY"))

# SimLP client
simlp_client = SimLPClient(
    reference_rules_dir=project_root / "data" / "ground_truth" / "msa",
    log_dir=project_root / "notebooks" / "logs"
)

print("✓ All components configured")

TypeError: SimLPClient.__init__() got an unexpected keyword argument 'reference_dir'

### 4.2 Create Orchestrator with Memory

In [ ]:
# Create a fresh memory for this demonstration
demo_memory = RuleMemory()

# Create orchestrator with memory
orchestrator = LoopOrchestrator(
    prompt_builder=prompt_builder,
    llm_provider=llm_provider,
    simlp_client=simlp_client,
    config=config,
    rule_memory=demo_memory,  # Pass memory instance
    verbose=True
)

print(f"✓ Orchestrator created with memory ({len(demo_memory)} entries)")

---
## 5. Example 1: Basic Activity (No Prerequisites)

First, generate rules for a foundational activity with no dependencies:

In [ ]:
# Run orchestrator for 'gap' activity
result_gap = orchestrator.run(
    domain="MSA",
    activity="gap",
    prerequisites=None  # No prerequisites
)

print(f"\n{'='*80}")
print("RESULT SUMMARY")
print(f"{'='*80}")
print(f"Activity: gap")
print(f"Converged: {result_gap.summary['converged']}")
print(f"Best Score: {result_gap.summary['best_score']:.4f}")
print(f"Iterations: {result_gap.summary['iterations_used']}")
print(f"Total Tokens: {result_gap.summary['total_tokens']}")

In [ ]:
# Check memory after run
print(f"Memory now has {len(demo_memory)} entries")
print(f"Fluents in memory: {demo_memory.list_fluents()}")

# Retrieve the stored entry
gap_entry = demo_memory.get("gap")
if gap_entry:
    print(f"\n✓ 'gap' successfully stored!")
    print(f"  Score: {gap_entry.score}")
    print(f"  Domain: {gap_entry.metadata.get('domain')}")
    print(f"  Prerequisites: {gap_entry.metadata.get('prerequisites')}")

---
## 6. Example 2: Composite Activity (With Prerequisites)

Now generate rules for a composite activity that depends on previously generated rules:

In [ ]:
# Run orchestrator for 'rendezVous' which depends on 'gap'
result_rdv = orchestrator.run(
    domain="MSA",
    activity="rendezVous",
    prerequisites=["gap"]  # Requires 'gap' rules
)

print(f"\n{'='*80}")
print("RESULT SUMMARY")
print(f"{'='*80}")
print(f"Activity: rendezVous")
print(f"Converged: {result_rdv.summary['converged']}")
print(f"Best Score: {result_rdv.summary['best_score']:.4f}")
print(f"Iterations: {result_rdv.summary['iterations_used']}")
print(f"Total Tokens: {result_rdv.summary['total_tokens']}")

In [ ]:
# Check memory after second run
print(f"Memory now has {len(demo_memory)} entries")
print(f"Fluents in memory: {demo_memory.list_fluents()}")

# Retrieve the stored entry
rdv_entry = demo_memory.get("rendezVous")
if rdv_entry:
    print(f"\n✓ 'rendezVous' successfully stored!")
    print(f"  Score: {rdv_entry.score}")
    print(f"  Prerequisites: {rdv_entry.metadata.get('prerequisites')}")
    print(f"\n  Rules preview:")
    print(f"  {rdv_entry.rules[:200]}...")

---
## 7. Memory Queries and Inspection

Explore the stored rules and metadata:

In [ ]:
# Get all entries as a DataFrame for easy viewing
entries = demo_memory.get_all()

df = pd.DataFrame([
    {
        "name": e.name,
        "score": e.score,
        "domain": e.metadata.get("domain"),
        "prerequisites": ", ".join(e.metadata.get("prerequisites", [])),
        "converged": e.metadata.get("converged"),
        "rules_length": len(e.rules),
        "created_at": e.created_at.strftime("%Y-%m-%d %H:%M:%S")
    }
    for e in entries
])

print("Memory Contents:")
print(df.to_string(index=False))

In [ ]:
# Get formatted rules for multiple fluents (for prompt injection)
formatted_multi = demo_memory.get_formatted_rules(
    names=["gap", "rendezVous"],
    include_description=True
)

print("Formatted rules for ['gap', 'rendezVous']:")
print(formatted_multi[:500])
print("...")

---
## 8. Updating Memory Entries

Update existing entries with new information:

In [ ]:
# Update the 'gap' entry with additional metadata
demo_memory.update(
    name="gap",
    metadata={
        "domain": "MSA",
        "prerequisites": [],
        "converged": True,
        "best_iteration": 1,
        "reviewed": True,
        "review_date": datetime.now().isoformat()
    }
)

# Verify update
updated_entry = demo_memory.get("gap")
print(f"Updated metadata for 'gap':")
print(f"  Reviewed: {updated_entry.metadata.get('reviewed')}")
print(f"  Review Date: {updated_entry.metadata.get('review_date')}")

---
## 9. Memory Serialization

Save and load memory for persistence:

In [ ]:
# Export memory to dictionary
memory_dict = demo_memory.export_to_dict()

print(f"Exported memory structure:")
print(f"  Entries: {len(memory_dict['entries'])}")
print(f"  Exported at: {memory_dict['exported_at']}")
print(f"\n  Entry keys: {list(memory_dict['entries'].keys())}")

In [ ]:
# Create a new memory and import
new_memory = RuleMemory()
new_memory.import_from_dict(memory_dict)

print(f"✓ Imported into new memory")
print(f"  Entries: {len(new_memory)}")
print(f"  Fluents: {new_memory.list_fluents()}")

In [ ]:
# Save to JSON file
import json

output_path = project_root / "notebooks" / "logs" / "memory_export.json"
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, 'w') as f:
    json.dump(memory_dict, f, indent=2)

print(f"✓ Memory exported to: {output_path}")

---
## 10. Advanced: Filtering by Metadata

Query memory by metadata attributes:

In [ ]:
# Get all entries for a specific domain
all_entries = demo_memory.get_all()
msa_entries = [
    e for e in all_entries
    if e.metadata.get("domain") == "MSA"
]

print(f"MSA domain entries: {[e.name for e in msa_entries]}")

In [ ]:
# Get entries with no prerequisites (foundational fluents)
foundational = [
    e for e in all_entries
    if not e.metadata.get("prerequisites")
]

print(f"Foundational fluents: {[e.name for e in foundational]}")

In [ ]:
# Get entries with high scores (>= 0.95)
high_quality = [
    e for e in all_entries
    if e.score and e.score >= 0.95
]

print(f"High-quality rules (score >= 0.95):")
for e in high_quality:
    print(f"  - {e.name}: {e.score:.4f}")

---
## 11. Complete Workflow Example

Here's a complete workflow showing how to build a dependency chain:

In [ ]:
# Create a fresh orchestrator and memory for complete example
workflow_memory = RuleMemory()
workflow_orchestrator = LoopOrchestrator(
    prompt_builder=prompt_builder,
    llm_provider=llm_provider,
    simlp_client=simlp_client,
    config=config,
    rule_memory=workflow_memory,
    verbose=False  # Less verbose for cleaner output
)

# Define activity dependency chain
activities = [
    {"name": "gap", "prerequisites": None},
    {"name": "lowSpeed", "prerequisites": None},
    {"name": "stopped", "prerequisites": ["lowSpeed"]},
    {"name": "rendezVous", "prerequisites": ["gap", "lowSpeed"]}
]

print("Building activity dependency chain...\n")
results = {}

for activity in activities:
    print(f"Processing: {activity['name']}")
    print(f"  Prerequisites: {activity['prerequisites'] or 'None'}")
    
    result = workflow_orchestrator.run(
        domain="MSA",
        activity=activity["name"],
        prerequisites=activity["prerequisites"]
    )
    
    results[activity["name"]] = result
    
    print(f"  ✓ Score: {result.summary['best_score']:.4f}")
    print(f"  ✓ Converged: {result.summary['converged']}")
    print(f"  ✓ Iterations: {result.summary['iterations_used']}")
    print()

print(f"{'='*80}")
print(f"Workflow complete! Memory has {len(workflow_memory)} entries")

In [ ]:
# Summary table
summary_df = pd.DataFrame([
    {
        "activity": name,
        "score": result.summary['best_score'],
        "converged": result.summary['converged'],
        "iterations": result.summary['iterations_used'],
        "tokens": result.summary['total_tokens']
    }
    for name, result in results.items()
])

print("Workflow Summary:")
print(summary_df.to_string(index=False))

---
## 12. Key Takeaways

### Memory Module Benefits:
1. **Reusability**: Store validated rules for reuse across activities
2. **Dependency Management**: Build complex activities on simpler prerequisites
3. **Efficiency**: Avoid regenerating foundational rules
4. **Traceability**: Track metadata (scores, domains, dependencies)
5. **Persistence**: Export/import for long-term storage

### Best Practices:
- Start with foundational activities (no prerequisites)
- Build complex activities incrementally
- Use metadata to track domains and dependencies
- Export memory periodically for backup
- Query memory to understand rule relationships

### Next Steps:
- Implement dependency graphs for automatic ordering
- Add validation for prerequisite availability
- Extend metadata for version tracking
- Create visualizations of activity dependencies

---
## 13. Cleanup

Clear memory if needed:

In [ ]:
# Clear specific entry
# demo_memory.remove("gap")

# Clear all entries
# demo_memory.clear()

print("Notebook complete! 🎉")